In [3]:
from pandas.io import data, wb
import datetime
import os.path
import math
import pandas as pd
import numpy

def update_ohlc(symbol):
    if (not os.path.isfile('data/'+symbol+'-ohlc.csv')):
        df_train = data.DataReader(symbol, 'yahoo', datetime.datetime(1900, 2, 1), datetime.datetime.now())

    else:
        df_train = pd.read_csv('data/'+symbol+'-ohlc.csv', index_col='Date', parse_dates=True)
        start = df_train.index[-1] + datetime.timedelta(days=1)
        print(start, datetime.datetime.now())

        if(start <= datetime.datetime.now()):
            new_data = data.DataReader(symbol, 'yahoo', start, datetime.datetime.now() )
            print(new_data)
            df_train = df_train.append(new_data)

    df_train.to_csv('data/'+symbol+'-ohlc.csv')
    
    if (not os.path.isfile('data/'+symbol+'-dividens.csv')):
        df_train = data.DataReader(symbol, 'yahoo', datetime.datetime(1900, 2, 1), datetime.datetime.now())
symbols = ['spy', 'qqq','iwm', 'eem', 'aapl', 'ibm', 'fb','goog', 'amzn','gs','intc', 'dva',
           'noah', 'lc', 'renn', 'tsla','hog', '600519.SS']
#symbols = ['fcx']
for symbol in symbols:
    update_ohlc(symbol)
print("update completed.")

2016-04-16 00:00:00 2016-04-18 23:22:15.160375
                  Open        High    Low       Close    Volume   Adj Close
Date                                                                       
2016-04-18  207.779999  209.279999  207.0  209.240005  75277700  209.240005
2016-04-16 00:00:00 2016-04-18 23:22:18.338110
              Open        High     Low       Close    Volume   Adj Close
Date                                                                    
2016-04-18  110.32  111.269997  110.25  111.230003  16618800  111.230003
2016-04-16 00:00:00 2016-04-18 23:22:20.442434
                  Open        High         Low       Close    Volume  \
Date                                                                   
2016-04-18  111.980003  113.389999  111.870003  113.300003  19114500   

             Adj Close  
Date                    
2016-04-18  113.300003  
2016-04-16 00:00:00 2016-04-18 23:22:21.538508
                 Open       High        Low  Close    Volume  Adj Close
D